In [1]:
#连接usb端口
%serialconnect to --port=/dev/user_voice --baud=115200

Connecting to --port=/dev/user_voice --baud=115200 
Ready.


In [2]:
import time
from Maix import GPIO, I2S
from fpioa_manager import fm

In [3]:
# user setting
sample_rate = 16000 #采样率
record_time = 4  # 录音时间，单位s

# fm.register(8,  fm.fpioa.GPIO0, force=True)
# wifi_en = GPIO(GPIO.GPIO0, GPIO.OUT)
# wifi_en.value(0)

# 定义麦克风引脚
fm.register(20, fm.fpioa.I2S0_IN_D0, force=True)
fm.register(18, fm.fpioa.I2S0_SCLK, force=True)
fm.register(19, fm.fpioa.I2S0_WS, force=True)

# fm.register(30,fm.fpioa.I2S0_WS, force=True)    # dock
# fm.register(32,fm.fpioa.I2S0_SCLK, force=True)  # dock

In [4]:
rx = I2S(I2S.DEVICE_0)  # 创建录音设备（麦克风）I2S
rx.channel_config(rx.CHANNEL_0, rx.RECEIVER, align_mode=I2S.STANDARD_MODE) # 配置通道
rx.set_sample_rate(sample_rate) # 设置采样率
print(rx)

[MAIXPY]i2s0:(sampling rate=16003, sampling points=1024)
[MAIXPY]channle0:(resolution=2, cycles=2, align_mode=1, mode=1)
[MAIXPY]channle1:(resolution=0, cycles=0, align_mode=0, mode=0)
[MAIXPY]channle2:(resolution=0, cycles=0, align_mode=0, mode=0)
[MAIXPY]channle3:(resolution=0, cycles=0, align_mode=0, mode=0)



In [5]:
from speech_recognizer import asr


# 定义声学模型(继承asr模块的辅助类)，与烧录的模型对应
class maix_asr(asr):
    asr_vocab = ["lv", "shi", "yang", "chun", "yan", "jing", "da", "kuai", "wen", "zhang", "de", "di", "se", "si",
                 "yue", "lin", "luan", "geng", "xian", "huo", "xiu", "mei", "yi", "ang", "ran", "ta", "jin", "ping",
                 "yao", "bu", "li", "liang", "zai", "yong", "dao", "shang", "xia", "fan", "teng", "dong", "she", "xing",
                 "zhuang", "ru", "hai", "tun", "zhi", "tou", "you", "ling", "pao", "hao", "le", "zha", "zen", "me",
                 "zheng", "cai", "ya", "shu", "tuo", "qu", "fu", "guang", "bang", "zi", "chong", "shui", "cuan", "ke",
                 "shei", "wan", "hou", "zhao", "jian", "zuo", "cu", "hei", "yu", "ce", "ming", "dui", "cheng", "men",
                 "wo", "bei", "dai", "zhe", "hu", "jiao", "pang", "ji", "lao", "nong", "kang", "yuan", "chao", "hui",
                 "xiang", "bing", "qi", "chang", "nian", "jia", "tu", "bi", "pin", "xi", "zou", "chu", "cun", "wang",
                 "na", "ge", "an", "ning", "tian", "xiao", "zhong", "shen", "nan", "er", "ri", "zhu", "xin", "wai",
                 "luo", "gang", "qing", "xun", "te", "cong", "gan", "lai", "he", "dan", "wei", "die", "kai", "ci", "gu",
                 "neng", "ba", "bao", "xue", "shuai", "dou", "cao", "mao", "bo", "zhou", "lie", "qie", "ju", "chuan",
                 "guo", "lan", "ni", "tang", "ban", "su", "quan", "huan", "ying", "a", "min", "meng", "wu", "tai",
                 "hua", "xie", "pai", "huang", "gua", "jiang", "pian", "ma", "jie", "wa", "san", "ka", "zong", "nv",
                 "gao", "ye", "biao", "bie", "zui", "ren", "jun", "duo", "ze", "tan", "mu", "gui", "qiu", "bai", "sang",
                 "jiu", "yin", "huai", "rang", "zan", "shuo", "sha", "ben", "yun", "la", "cuo", "hang", "ha", "tuan",
                 "gong", "shan", "ai", "kou", "zhen", "qiong", "ding", "dang", "que", "weng", "qian", "feng", "jue",
                 "zhuan", "ceng", "zu", "bian", "nei", "sheng", "chan", "zao", "fang", "qin", "e", "lian", "fa", "lu",
                 "sun", "xu", "deng", "guan", "shou", "mo", "zhan", "po", "pi", "gun", "shuang", "qiang", "kao", "hong",
                 "kan", "dian", "kong", "pei", "tong", "ting", "zang", "kuang", "reng", "ti", "pan", "heng", "chi",
                 "lun", "kun", "han", "lei", "zuan", "man", "sen", "duan", "leng", "sui", "gai", "ga", "fou", "kuo",
                 "ou", "suo", "sou", "nu", "du", "mian", "chou", "hen", "kua", "shao", "rou", "xuan", "can", "sai",
                 "dun", "niao", "chui", "chen", "hun", "peng", "fen", "cang", "gen", "shua", "chuo", "shun", "cha",
                 "gou", "mai", "liu", "diao", "tao", "niu", "mi", "chai", "long", "guai", "xiong", "mou", "rong", "ku",
                 "song", "che", "sao", "piao", "pu", "tui", "lang", "chuang", "keng", "liao", "miao", "zhui", "nai",
                 "lou", "bin", "juan", "zhua", "run", "zeng", "ao", "re", "pa", "qun", "lia", "cou", "tie", "zhai",
                 "kuan", "kui", "cui", "mie", "fei", "tiao", "nuo", "gei", "ca", "zhun", "nie", "mang", "zhuo", "pen",
                 "zun", "niang", "suan", "nao", "ruan", "qiao", "fo", "rui", "rao", "ruo", "zei", "en", "za", "diu",
                 "nve", "sa", "nin", "shai", "nen", "ken", "chuai", "shuan", "beng", "ne", "lve", "qia", "jiong", "pie",
                 "seng", "nuan", "nang", "miu", "pou", "cen", "dia", "o", "zhuai", "yo", "dei", "n", "ei", "nou", "bia",
                 "eng", "den", "_"]

    def get_asr_list(string='xiao-ai-fas-tong-xue'):
        return [__class__.asr_vocab.index(t) for t in string.split('-') if t in __class__.asr_vocab]

    def get_asr_string(listobj=[117, 214, 257, 144]):
        return '-'.join([__class__.asr_vocab[t] for t in listobj if t < len(__class__.asr_vocab)])

    def unit_test():
        print(__class__.get_asr_list('xiao-ai'))
        print(__class__.get_asr_string(__class__.get_asr_list('xiao-ai-fas-tong-xue')))

    def config(self, sets):
        self.set([(sets[key], __class__.get_asr_list(key)) for key in sets])

    def recognize(self):
        res = self.result()
        # print(tmp)
        if res != None:
            sets = {}
            for tmp in res:
                sets[__class__.get_asr_string(tmp[1])] = tmp[0]
                # print(tmp[0], get_asr_string(tmp[1]))
            return sets
        return None

In [6]:
from machine import Timer  # 导入Timer

def on_timer(timer):
    # print("time up:",timer)
    # print("param:",timer.callback_arg())
    timer.callback_arg().state()

try:
    # default: maix dock / maix duino set shift=0库
    t = maix_asr(0x500000, I2S.DEVICE_0, 3, shift=1)  # 创建声学模型对象，shift=1使能麦克风
    tim = Timer(Timer.TIMER0, Timer.CHANNEL0, mode=Timer.MODE_PERIODIC, period=64, callback=on_timer, arg=t) # 创建定时器对象
    tim.start() #启动定时器

    # for i in range(50):
    # time.sleep_ms(100)
    # t.stop()
    # for i in range(50):
    # time.sleep_ms(100)
    # t.run()
# 配置识别词条(关键字，匹配阈值)
    t.config({  
        'xiao-ai': 0.3,
        'hao-de': 0.2,
        'ni-hao': 0.3,
    })

    print(t.get())

    while True:
        # time.sleep(1)
        tmp = t.recognize()  #识别语音
        # print(tmp)
        if tmp != None:
            print(tmp)  # 输出结果
except Exception as e:
    print(e)
finally:
    tim.stop()
    t.__del__()
    del t

[(0.3, [117, 214]), (0.3, [157, 51]), (0.2, [51, 10])]
................................{'xiao-ai': 0.6663998}
{'xiao-ai': 0.666919}
{'xiao-ai': 0.666919}
{'xiao-ai': 0.6870356}
{'xiao-ai': 0.7179453}
{'xiao-ai': 0.7179453}
{'xiao-ai': 0.7179453}
{'ni-hao': 1.209417}
{'ni-hao': 1.209417}
{'ni-hao': 1.209417}
{'ni-hao': 1.209417}
{'ni-hao': 1.540446}
{'ni-hao': 1.540446}
{'ni-hao': 1.540446}
{'ni-hao': 0.9540892}
{'ni-hao': 0.9540892}
{'ni-hao': 0.9540892}
..

*** Sending Ctrl-C

speech_asr_make_del __del__


Traceback (most recent call last):
  File "<stdin>", line 40, in <module>
  File "<stdin>", line 36, in <module>
  File "<stdin>", line 33, in <module>
KeyboardInterrupt: 
